In [28]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
!wget https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!dpkg -i google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
!apt-get install -f
!apt-get -y install -qq fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
--2019-04-21 16:41:54--  https://launchpad.net/~alessandro-strada/+archive/ubuntu/google-drive-ocamlfuse-beta/+build/15331130/+files/google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb
Resolving launchpad.net (launchpad.net)... 91.189.89.223, 91.189.89.222, 2001:67c:1560:8003::8004, ...
Connecting to launchpad.net (launchpad.net)|91.189.89.223|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2019-04-21 16:41:54 ERROR 404: Not Found.

dpkg: error: cannot access archive 'google-drive-ocamlfuse_0.7.0-0ubuntu1_amd64.deb': No such file or directory
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.
··········


In [29]:
!mkdir -p drive
!google-drive-ocamlfuse drive

fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [30]:
!pip install tensorflow-gpu==2.0.0-alpha0

In [0]:
import cv2
import tensorflow as tf
import numpy as np
from tqdm import tqdm
from sklearn.datasets import load_files
import matplotlib.pyplot as plt

In [0]:
def load_data(path):
    data = load_files(path)
    files = np.array(data['filenames'])
    return files

In [0]:
def ext_img(img):
    imag = cv2.imread(img)
    imag = cv2.cvtColor(imag,cv2.COLOR_BGR2GRAY)
    imag = imag/255
    imag = np.expand_dims(imag,axis=2)
    return imag

In [0]:
def tensor_4d(fil):
    lis = [ext_img(im) for im in tqdm(fil)]
    return np.stack(lis,axis=0)

In [0]:
fold_path = 'drive/datasets/shapes'

In [0]:
img_files = load_data(fold_path)

In [37]:
imgs = tensor_4d(img_files)

100%|██████████| 300/300 [00:00<00:00, 446.72it/s]


In [38]:
imgs.shape

(300, 28, 28, 1)

In [0]:
class Encoder(tf.keras.Model):
    
    def __init__(self):
        super(Encoder,self).__init__()
        self.conv8 = tf.keras.layers.Conv2D(8,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.conv16 = tf.keras.layers.Conv2D(16,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.conv32 = tf.keras.layers.Conv2D(32,3,activation=tf.nn.relu,strides=(1,1),padding='SAME')
        self.max_pool_1 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))
        self.max_pool_2 = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=(2,2))
        
    def call(self,inp):
        x = self.conv32(inp)
        x = self.max_pool_1(x)
        x = self.conv16(x)
        x = self.max_pool_2(x)
        x = self.conv8(x)
        return x
        
        
        

In [0]:
inp = tf.keras.Input(shape=(28,28,1))
enc = Encoder()

In [41]:
enc(inp)

<tf.Tensor 'encoder_2/conv2d_6/Relu:0' shape=(None, 7, 7, 8) dtype=float32>

In [0]:
class Decoder(tf.keras.Model):
    
    def __init__(self):
        super(Decoder,self).__init__()
        self.convtr8 = tf.keras.layers.Conv2DTranspose(8,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr16 = tf.keras.layers.Conv2DTranspose(16,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr32 = tf.keras.layers.Conv2DTranspose(32,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.convtr1 = tf.keras.layers.Conv2DTranspose(1,3,strides=(1,1),padding='SAME',activation=tf.nn.relu)
        self.upsample_1 = tf.keras.layers.UpSampling2D(size=(2,2))
        self.upsample_2 = tf.keras.layers.UpSampling2D(size=(2,2))
        
    def call(self,inp):
        x = self.convtr8(inp)
        x = self.upsample_1(x)
        x = self.convtr16(x)
        x = self.upsample_2(x)
        x = self.convtr32(x)
        x = self.convtr1(x)
        return x
        
        

In [0]:
dec = Decoder()

In [0]:
op = enc(inp)
oc = dec(op)

In [0]:
def loss(ips,ops):
    mse_loss = tf.keras.losses.MeanSquaredError()
    mse = mse_loss(ips,ops)
    return mse

In [0]:
def grad(ips):
    with tf.GradientTape() as tape:
        ip = tf.keras.Input(shape=(28,28,1),dtype='float32')
        encode = Encoder()
        decode = Decoder()
        encoded = encode(ips)
        decoded = decode(encoded)
        err = loss(ips,decoded)
        #'''
        trainable_vars = [encode.conv8.trainable_variables,encode.conv16.trainable_variables,encode.conv32.trainable_variables,
                          encode.max_pool_1.trainable_variables,encode.max_pool_2.trainable_variables,
                          decode.convtr8.trainable_variables,decode.convtr16.trainable_variables,decode.convtr32.trainable_variables,
                          decode.convtr1.trainable_variables,decode.upsample_1.trainable_variables,decode.upsample_2.trainable_variables]#'''
        
        #trainable_vars = encode.trainable_variables
        
    return err, tape.gradient(err,trainable_vars),trainable_vars

In [0]:
optimiser = tf.keras.optimizers.Adam(learning_rate=0.001)
        

In [96]:
for epoch in range(30):
    for start,end in zip(range(0,len(imgs),10),range(10,len(imgs),10)):
        xtr = imgs[start:end]
        loss_values,grad,tra_var = grad(xtr)
        print(loss_values)
        print(grad)
        optimiser.apply_gradients(zip(grad,tra_var))

tf.Tensor(0.8798657059669495, shape=(), dtype=float64)
[[<tf.Tensor: id=5665, shape=(3, 3, 16, 8), dtype=float64, numpy=
array([[[[ 5.05388676e-03,  3.51066786e-03,  3.67108856e-03, ...,
          -1.30052574e-03, -4.27169572e-03, -1.64720714e-03],
         [ 8.07797787e-03,  5.63405136e-03,  5.76840182e-03, ...,
          -1.95863530e-03, -6.58790169e-03, -2.40372404e-03],
         [ 8.44503241e-04,  7.61954173e-04,  7.45149851e-04, ...,
          -3.40191419e-04, -6.47600031e-04, -1.39812519e-04],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
           0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
         [ 8.92849461e-04,  4.96499933e-04,  6.43067974e-04, ...,
          -1.81723759e-04, -7.17555842e-04, -2.90856401e-04],
         [ 2.60792432e-03,  1.30953127e-03,  1.37886646e-03, ...,
          -2.36023270e-04, -1.82802164e-03, -9.02907198e-04]],

        [[ 4.44150735e-03,  3.95255101e-03,  4.18279333e-03, ...,
          -2.30064554e-03, -4

AttributeError: ignored

In [89]:
dec.upsample_1.trainable_variables

[]